In [ ]:
import numpy as np
import sympy


In [ ]:
def chemical_balancing(self,p):
        """Calculate yield coefficients from p for the following metabolic pathways:

        1. (ox) : Oxidative glucose consumption
        2. (red) : Reductive glucose consumption
        3. (et) : Ethanol consumption
        4. (glyc) : Glycerol consumption
        5. (m) : Maintenence metabolism

        :param p: Parameters 
        :type p: lmfit.parameter.Parameters object

        :return: yields: Yields coefficients of metabolic pathways
        """
        
        #Order:         C,      H,      O,      N numbers of atoms in the molecule e.g. glucose : C6H12O6N0
        gluc = np.array([6.0,   12.0,   6.0,    0.0])
        O2 = np.array([0.0,     0.0,     2.0,   0.0])
        NH3 = np.array([0.0,    3.0,    0.0,    1.0])
        biomass = np.array([1.0,p["HX"].value, p["OX"].value, p["NX"].value])
        CO2 = np.array([1.0,    0.0,    2.0,    0.0])
        H2O = np.array([0.0,    2.0,    1.0,    0.0])
        etoh = np.array([2.0,   6.0,    1.0,    0.0])
        glyc = np.array([3.0,   8.0,    3.0,    0.0])


        #Required yields to solve linear equation system taken from p
        yields = {"Yxs": p["Yxs"].value, 
                  "Yxp" : p["Yxp"].value, 
                  "Yxsmain" : p["Yxsmain"].value}
        
        MW_element_dict = {"C": 12.011, "H": 1.0079, "O": 15.999, "N": 14.007}        #molar masses of elements
        molecule = {"gluc": gluc, "O2": O2, "NH3" : NH3, "biomass": biomass, "CO2" : CO2, "H2O":  H2O, "etoh": etoh, "glyc": glyc} #molecules dict
        
        MW = {}           #MW = molar weights: dict with masses of molecules
        for key, mol in molecule.items():
            molecule_MW_array = ([])
            for vectorvalue, weight in zip (mol, MW_element_dict.values()):
                vw = vectorvalue*weight
                molecule_MW_array= np.append(molecule_MW_array, vw)
            MW[key] = sum(molecule_MW_array)
            
        NX1 = p["NX"].value #NX1 because NX is reserved for the symbol letter NX from sympy to solve the equation
       
        #1. oxidative glucose consumption: gluc+ a*O2 + b*NX*NH3 = b*biomass + c*CO2 + d*H2O 
        a,b,c,d, NX = symbols("a b c d NX")         #set symbols to solve equation
        Yxs_ox = p["Yxs_ox"].value
        b1 = Yxs_ox* MW["gluc"]/MW["biomass"]     #calculate stoichiometric coefficient from mass related yield coefficient

        eqOx_list = []
        for num in range(3):        #Set up reaction equations three times because three unknowns are present
            eqOx = Eq(gluc[num]+ a*O2[num]+ b*NX*NH3[num], b*biomass[num]+ c*CO2[num]+ d*H2O[num])
            eqOx = eqOx.subs({b: b1, NX: NX1})
            eqOx_list.append(eqOx)
        
        solution_Ox = sp.solve(eqOx_list, (a, c, d), dict= True)        #solve equation system
        a1, c1, d1 = np.float(solution_Ox[0][a]), np.float(solution_Ox[0][c]), np.float(solution_Ox[0][d])  #assign results to variables

        YCO2x_ox = c1/b1 * MW["CO2"]/MW["biomass"]
        YCO2s_ox = c1/1 * MW["CO2"]/MW["gluc"]
        YO2s_ox = a1/1 * MW["O2"]/MW["gluc"]
        
        yields["YCO2x_ox"], yields["YCO2s_ox"], yields["YO2s_ox"] = YCO2x_ox, YCO2s_ox, YO2s_ox 
        
         #5. maintenance metabolism : gluc + 6*O2 = 6*CO2 + 6*H2O

        YCO2s_m = 6 * MW["CO2"]/MW["gluc"]
  

        yields["YCO2s_m"] = YCO2s_m
        
        return yields